In [ ]:
# mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# change current working directory
import os
os.chdir('/content/drive/MyDrive/AI/6-B-AI/')

In [ ]:
# check we can see the dataset
os.path.exists('FaceDatabaseATT/')

In [ ]:
# train an ANN architecture on the MNIST image dataset using TensorFlow

# this is a development of the final code example from the last lab - take some time to review that example
# if you haven't already looked at it.

# data preparation steps

# Importing the packages we use
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

# Loading the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')

# Examples and labels
examples = mnist['data']
labels = mnist['target']

# Convert class label strings to integers (required for fast tf operation)
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Splitting into testing examples and training examples with corresponding labels
train_examples, test_examples, train_labels, test_labels = train_test_split(examples, labels, test_size=0.25, random_state=99)

# Standardise our predictive features
scaler = StandardScaler()
train_examples = scaler.fit_transform(train_examples)
test_examples = scaler.transform(test_examples)

# model training and model evaluation steps

# Re-seed for reproducible results
tf.random.set_seed(99)
# Build a feed-forward network
model = tf.keras.Sequential()
# Input layer sizing based on the number of pixels in our images
model.add(tf.keras.layers.InputLayer(input_shape=[784]))
# Then, keeping things relatively simple/shallow initially, 3 hidden layers with 100 nodes and relu activations
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
model.add(tf.keras.layers.Dense(100, activation="relu"))
# And an output layer using a "softmax" function to ensure we get class probabilities that sum to 1
model.add(tf.keras.layers.Dense(10, activation="softmax"))

# Compile the model, asking to use cross-entropy as our cost function, and SGD for gradient descent
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd", metrics=["accuracy"])

# Call the model's fitting algorithm, passing in our training examples and training labels, and retaining information about the descent
history = model.fit(train_examples, train_labels, epochs=10)

# Visualise the gradient descent process
plt.figure()
plt.plot(history.history['loss'], label='Training Cost')
plt.title('Cost')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.legend()
plt.show()

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)
# Convert probabilities to predicted class labels (they are arrays of class probability scores by default)
predictions = np.argmax(predictions, axis=1)

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# We've used familiar code above, but note tf has it's own evaluation helper
#model.evaluate(test_examples, test_labels)

# WARNING: though this is a relatively small dataset, containing tiny images...
# ...the code will still take a while to execute (and may take longer if you...
# ...experiment with other architectures...)

In [ ]:
# try some feature extraction operations on an image:

import matplotlib.pyplot as plt
from skimage import io, color, feature, filters, exposure
from scipy import ndimage as ndi
from skimage.transform import resize
import numpy as np

# Load the image
image = io.imread('peppers.png')
print("Original image shape:", image.shape)

# Show the colour image
plt.imshow(image)
plt.title('Original Image')
plt.show()

# Convert to grayscale
gray_image = color.rgb2gray(image)
print("Grayscale image shape:", gray_image.shape)

# Show the grayscale image
plt.imshow(gray_image, cmap='gray')
plt.title('Grayscale Image')
plt.show()

# Convolution with a vertical edge detection filter
vertical_filter = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
edges_vertical = ndi.convolve(gray_image, vertical_filter)

# Show the vertical edges
plt.imshow(edges_vertical, cmap='gray')
plt.title('Vertical Edges')
plt.show()

# Convolution with a horizontal edge detection filter
horizontal_filter = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
edges_horizontal = ndi.convolve(gray_image, horizontal_filter)

# Show the horizontal edges
plt.imshow(edges_horizontal, cmap='gray')
plt.title('Horizontal Edges')
plt.show()

# Calculate the gradient magnitude
edges_magnitude = np.sqrt(edges_vertical**2 + edges_horizontal**2)

# Normalize the gradient magnitude to the range [0, 1] for visualization
edges_magnitude_normalized = edges_magnitude / np.max(edges_magnitude)

# Show the gradient magnitude
plt.imshow(edges_magnitude_normalized, cmap='gray')
plt.title('Gradient Magnitude')
plt.show()

# HOG feature extraction and visualization
fd, hog_image = feature.hog(gray_image, orientations=8, pixels_per_cell=(16, 16), visualize=True, block_norm='L2')

# Show HOG image
plt.imshow(hog_image, cmap='gray')
plt.title('HOG Features Visualization')
plt.show()

In [ ]:
# train a model using HOG features on the FaceDatabaseATT dataset

# note: here we also work through an example of loading images from individual local files
# (in contrast to loading them up from a pre-processed source, as with MNIST previously)
# - this is a reusable template that you can apply with other similarly-structured image datasets

import os
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from skimage import io, transform
from skimage.feature import hog
import time

def extract_hog_features(images, cell_size=(16, 16)):
    hog_features = []
    for image in images:
        # Calculate the number of cells that fit into the image
        n_cells_y = image.shape[0] // cell_size[0]  # number of cells vertically
        n_cells_x = image.shape[1] // cell_size[1]  # number of cells horizontally

        # Set cells_per_block to cover the entire image (simplest lighting treatment)
        cells_per_block = (n_cells_y, n_cells_x)

        # Compute HOG features (standardising with a single unit L2 norm across the image (which is a single block))
        fd = hog(image, orientations=8, pixels_per_cell=cell_size, cells_per_block=cells_per_block, block_norm='L2', visualize=False, channel_axis=None)
        hog_features.append(fd)
    return np.array(hog_features)

# Support reading images from local storage (assuming classes separated by subfolders)
def load_images_and_labels(base_dir, target_size=None):
    images = []
    labels = []
    for folder in os.listdir(base_dir):
        folder_path = os.path.join(base_dir, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                img = io.imread(file_path)
                # though we won't use it here, support the option to resize images too
                if target_size:
                    img = transform.resize(img, target_size)
                images.append(img)
                labels.append(folder)
    return np.array(images), np.array(labels)

# load up the AT&T faces dataset
base_dir = 'FaceDatabaseATT'
images, labels = load_images_and_labels(base_dir)

# Extract HOG features from each image
hog_examples = extract_hog_features(images, cell_size=(16,16))

# Splitting into testing examples and training examples with corresponding labels
train_examples, test_examples, train_labels, test_labels = train_test_split(hog_examples, labels, test_size=0.25, random_state=99)

# model training and model evaluation steps

# Start timing
train_start_time = time.time()

# Create a NB model object
model = GaussianNB()

# Call the model's fitting algorithm, passing in our training examples and training labels
model.fit(train_examples, train_labels)

# Stop timing
train_end_time = time.time()

# Calculate the elapsed time for training
train_time = train_end_time - train_start_time
print("Training took", train_time, "seconds")

# Start timing for prediction
predict_start_time = time.time()

# Use the trained model to generate predictions for our testing examples
predictions = model.predict(test_examples)

# Stop timing for prediction
predict_end_time = time.time()

# Calculate the elapsed time for prediction
predict_time = predict_end_time - predict_start_time
print("Prediction took", predict_time, "seconds")

# Find the total number of model predictions that matched with the corresponding testing labels
correct_predictions = sum(predictions == test_labels)
# Calculate the model's accuracy: the fraction of predictions that were correct
accuracy = correct_predictions / len(test_labels)
# Display the accuracy as a single quantitative measure of overall performance
print("Accuracy:", accuracy, "(or", round(accuracy*100, 1), "%)")

# WARNING: this code will take a while to execute for the first time only...
# ... (and may take longer if you experiment with training other models...)

In [ ]:
# train an ANN architecture on the MNIST image dataset using PyTorch

# this is a reworking of the TensorFlow example from earlier on, to use PyTorch

# Importing the packages we use
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
from sklearn.datasets import fetch_openml
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Check if GPU is available and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Loading the MNIST dataset
mnist = fetch_openml('mnist_784', version=1, as_frame=False, parser='auto')
examples = mnist['data']
labels = mnist['target']

# Convert class label strings to integers
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Splitting the dataset
train_examples, test_examples, train_labels, test_labels = train_test_split(examples, labels, test_size=0.25, random_state=99)

# Standardise our predictive features
scaler = StandardScaler()
train_examples = scaler.fit_transform(train_examples)
test_examples = scaler.transform(test_examples)

# Re-seed for reproducible results
torch.manual_seed(99)

# Convert to PyTorch tensors and move them to the device
train_examples = torch.tensor(train_examples, dtype=torch.float32).to(device)
test_examples = torch.tensor(test_examples, dtype=torch.float32).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.long).to(device)
test_labels = torch.tensor(test_labels, dtype=torch.long).to(device)

# Create DataLoader for batch processing (and set the batch size)
train_dataset = TensorDataset(train_examples, train_labels)
train_loader = DataLoader(train_dataset, batch_size=64)

# model training and model evaluation steps

# Build a feed-forward network
model = nn.Sequential(
    # Input layer sizing based on the number of pixels in our images
    nn.Linear(784, 100),
    nn.ReLU(),
    # Then, keeping things relatively simple/shallow initially, 3 hidden layers with 100 nodes and relu activations
    nn.Linear(100, 100),
    nn.ReLU(),
    nn.Linear(100, 100),
    nn.ReLU(),
    # And an output layer using a "softmax" function to ensure we get class probabilities that sum to 1
    nn.Linear(100, 10)
).to(device)

# Asking to use cross-entropy as our cost function, and SGD for gradient descent
cost = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

# Call the model's fitting algorithm manually, one bactch at a time, and retaining information about the descent
num_epochs = 10
model.train()

# Initialize a list to store the average loss per epoch
epoch_losses = []

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = cost(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    avg_loss = running_loss / len(train_loader)
    epoch_losses.append(avg_loss)  # Store the average loss for this epoch
    print("Epoch", epoch+1, "/", num_epochs, "Loss:", avg_loss)

# Visualize training loss
plt.figure()
plt.plot(range(num_epochs), epoch_losses, label='Cost')  # Plot the stored losses
plt.title('Training Cost')
plt.xlabel('Epoch')
plt.ylabel('Cost')
plt.legend()
plt.show()

# Evaluating the model
model.eval()
with torch.no_grad():
    # Ensure test_examples are on the same device as the model
    test_examples = test_examples.to(device)
    outputs = model(test_examples)
    _, predicted = torch.max(outputs, 1)

    # Calculate correct predictions
    correct_predictions = torch.sum(predicted == test_labels).item()  # Convert to Python number with .item()

    # Calculate accuracy
    accuracy = correct_predictions / len(test_labels)

    # Print the accuracy
    print("Accuracy:", accuracy, "(or", round(accuracy * 100, 1), "%)")